In [ ]:
import sqlite3

conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()

In [73]:
import random
for i in range(10):
    print(wa

False
True
False
True
False
False
True
False
True
False


In [55]:
f = Foo()
print(f)

FOO


States.START
building!
homeview...
any to study?
Section view!
Front view!
Back view!
Front view!
Back view!
CORRECT!


True